In [ ]:
import numpy as np
import pandas as pd
from PyEMD import EMD
import matplotlib.pyplot as plt

def perform_emd_analysis(data, num_imfs=5):
    # Initialize EMD
    emd = EMD()
    
    # Perform EMD
    imfs = emd(data, max_imf=num_imfs)
    
    # Plot IMFs
    plt.figure(figsize=(12, 10))
    for i, imf in enumerate(imfs):
        plt.subplot(num_imfs + 1, 1, i + 1)
        plt.plot(imf)
        plt.title(f'IMF {i+1}')
    plt.tight_layout()
    plt.show()
    
    return imfs

def detect_abnormal_volatility(data, imfs, window_size=20, threshold=2):
    # Reconstruct signal using selected IMFs (e.g., excluding the trend)
    reconstructed = np.sum(imfs[:-1], axis=0)
    
    # Calculate rolling standard deviation
    rolling_std = pd.Series(reconstructed).rolling(window=window_size).std()
    
    # Detect abnormal volatility
    mean_std = rolling_std.mean()
    std_std = rolling_std.std()
    abnormal = rolling_std > (mean_std + threshold * std_std)
    
    # Plot results
    plt.figure(figsize=(12, 6))
    plt.plot(data, label='Original Data')
    plt.plot(reconstructed, label='Reconstructed (without trend)')
    plt.plot(rolling_std, label='Rolling Std Dev')
    plt.scatter(np.where(abnormal)[0], data[abnormal], color='red', label='Abnormal Volatility')
    plt.legend()
    plt.title('Volatility Analysis with EMD')
    plt.show()
    
    return abnormal

# Example usage (replace with your actual data)
np.random.seed(42)
time = np.linspace(0, 10, 1000)
volatility = np.abs(np.sin(time) + 0.5 * np.sin(5 * time) + 0.3 * np.random.randn(1000))

imfs = perform_emd_analysis(volatility)
abnormal_periods = detect_abnormal_volatility(volatility, imfs)

print(f"Detected {sum(abnormal_periods)} periods of abnormal volatility.")